### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [4]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [5]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [8]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [9]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [10]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [11]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [12]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [13]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [14]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], dtype=int64)

In [15]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [16]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [17]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [18]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [19]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [20]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.


<br> 
<br>
<br>
<br>
<br>
<br>

## <font color= "orane"> Solución </font>

#### <font color= "orange"> Punto 1 </font>

In [2]:
import random
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import fetch_20newsgroups

# Cargamos los datos

newsgroups_data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))


# Dividimos los datos en train y test

train_size = int(0.8 * len(newsgroups_data.data))
train_indices = random.sample(range(len(newsgroups_data.data)), train_size)
test_indices = list(set(range(len(newsgroups_data.data))) - set(train_indices))

newsgroups_train = {
    'data': [newsgroups_data.data[i] for i in train_indices],
    'target': [newsgroups_data.target[i] for i in train_indices],
    'target_names': newsgroups_data.target_names
}

newsgroups_test = {
    'data': [newsgroups_data.data[i] for i in test_indices],
    'target': [newsgroups_data.target[i] for i in test_indices],
    'target_names': newsgroups_data.target_names
}

print(f"Cantidad de documentos de entrenamiento: {len(newsgroups_train['data'])}")
print(f"Cantidad de documentos de prueba: {len(newsgroups_test['data'])}")


# Vectorización de los documentos

tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train['data'])

print(f'Tipo de X_train: {type(X_train)}')
print(f'Shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')


# Seleccionamos 5 documentos al azar y luego, medimos la similaridad

random.seed(37)  # Se agrega una semilla para que los resultados se puedan reproducir a la hora de probar el código
random_docs_indices = random.sample(range(len(newsgroups_train['data'])), 5)

for idx in random_docs_indices:
    
    print(f"\nDocumento original (índice {idx}):\n{newsgroups_train['data'][idx]}")
    print(f"Etiqueta: {newsgroups_train['target_names'][newsgroups_train['target'][idx]]}")

    # Se mide la similaridad coseno con todos los documentos de train
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    
    # Obtenemos los 5 documentos más similares (excluyendo el original)
    mostsim = np.argsort(cossim)[::-1][1:6]
    
    print("\nDocumentos más similares:")
    for i in mostsim:
        print(f"\nDocumento similar (índice {i}):\n{newsgroups_train['data'][i]}")
        print(f"Etiqueta: {newsgroups_train['target_names'][newsgroups_train['target'][i]]}")
        print(f"\n\nSimilaridad: {cossim[i]}")

Cantidad de documentos de entrenamiento: 15076
Cantidad de documentos de prueba: 3770
Tipo de X_train: <class 'scipy.sparse._csr.csr_matrix'>
Shape: (15076, 114642)
Cantidad de documentos: 15076
Tamaño del vocabulario (dimensionalidad de los vectores): 114642

Documento original (índice 11173):
Has anybody compiled VOGL-graphics library
for IBM-PC? I need to call it from MS-Fortran
but don't have MS-C to compile the sources.

Thanks for any help...

Etiqueta: comp.graphics

Documentos más similares:

Documento similar (índice 4382):
Three q's:

1) is it reliable?

2) how does it send the information from a MS Windows app over
the X11 protocol? Does it just draw everything as graphics into
one window, or does it use multiple windows and essentially work
more cleverly?

3) If I want to run MS Word, for example, remotely, do I have to
run a separate copy of MS Windows remotely, and then start MS
Word from that, or can MS Word be started remotely on its own?
Etiqueta: comp.windows.x


Simi

#### <font color= "orange"> Punto 2 </font>

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB, ComplementNB

# Se trabaja a partir de newsgroups_train y newsgroups_test que se han creado previamente

# A continuación creamos una función que entrenará y evaluará el modelo en función de los parámetros que se le pasen

def train_and_evaluate(vectorizer_params, model, model_name):
    
    # Vectorizamos

    print(f"\nVectorizamos documentos con los parámetros: {vectorizer_params}")
    tfidfvect = TfidfVectorizer(**vectorizer_params)
    X_train = tfidfvect.fit_transform(newsgroups_train['data'])
    X_test = tfidfvect.transform(newsgroups_test['data'])

    y_train = newsgroups_train['target']
    y_test = newsgroups_test['target']

    # Se entrena el modelo

    print(f"Entrenando modelo: {model_name}")
    model.fit(X_train, y_train)

    # Predicción y evaluación

    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='macro')
    print(f"F1-score (macro) para {model_name} con {vectorizer_params}: {f1}")

    return f1

# Parámetros a probar

vectorizer_params_list = [
    {'max_df': 0.75, 'min_df': 3, 'ngram_range': (1, 1)},
    {'max_df': 0.75, 'min_df': 2, 'ngram_range': (1, 2)},
    {'max_df': 0.75, 'min_df': 3, 'ngram_range': (1, 2)},
    {'max_df': 0.75, 'min_df': 2, 'ngram_range': (1, 1)},
    {'max_df': 0.85, 'min_df': 3, 'ngram_range': (1, 2)},
    {'max_df': 0.9, 'min_df': 1, 'ngram_range': (1, 1)},
    {'max_df': 0.8, 'min_df': 5, 'ngram_range': (1, 2)},
]

models = [
    (MultinomialNB(), "MultinomialNB"),
    (ComplementNB(), "ComplementNB")
]

# Probamos todas las combinaciones de vectorizador y modelo

best_f1 = 0
best_params = None
best_model_name = None

for vectorizer_params in vectorizer_params_list:
    for model, model_name in models:
        f1 = train_and_evaluate(vectorizer_params, model, model_name)
        if f1 > best_f1:
            best_f1 = f1
            best_params = vectorizer_params
            best_model_name = model_name

print(f"\nMejor F1-score: {best_f1} con {best_model_name} y parámetros {best_params}")


Vectorizamos documentos con los parámetros: {'max_df': 0.75, 'min_df': 3, 'ngram_range': (1, 1)}
Entrenando modelo: MultinomialNB
F1-score (macro) para MultinomialNB con {'max_df': 0.75, 'min_df': 3, 'ngram_range': (1, 1)}: 0.6734721654552351

Vectorizamos documentos con los parámetros: {'max_df': 0.75, 'min_df': 3, 'ngram_range': (1, 1)}
Entrenando modelo: ComplementNB
F1-score (macro) para ComplementNB con {'max_df': 0.75, 'min_df': 3, 'ngram_range': (1, 1)}: 0.7485544234458648

Vectorizamos documentos con los parámetros: {'max_df': 0.75, 'min_df': 2, 'ngram_range': (1, 2)}
Entrenando modelo: MultinomialNB
F1-score (macro) para MultinomialNB con {'max_df': 0.75, 'min_df': 2, 'ngram_range': (1, 2)}: 0.640663050175408

Vectorizamos documentos con los parámetros: {'max_df': 0.75, 'min_df': 2, 'ngram_range': (1, 2)}
Entrenando modelo: ComplementNB
F1-score (macro) para ComplementNB con {'max_df': 0.75, 'min_df': 2, 'ngram_range': (1, 2)}: 0.7614575076630652

Vectorizamos documentos con 

El mejor F1 Score se da para el modelo ComplementNB en donde:

- Ignora los términos que aparecen en más del 75% de los documentos
- Considera sólo los términos que aparecen en al menos 2 documentos
- Considera unigramas y bigramas

#### <font color= "orange"> Punto 3 </font>

In [9]:
# newsgroups_train y newsgroups_test ya están definidos en el punto 1

# Usamos los parámetros óptimos para vectorizar unigramas (palabras individuales) en los documentos

tfidfvect = TfidfVectorizer(max_df=0.75, min_df=2, ngram_range=(1, 1))
X_train = tfidfvect.fit_transform(newsgroups_train['data'])

# Transponemos  la matriz documento-término para obtener la matriz término-documento

X_train_T = X_train.T

# Obtenemos el vocabulario (términos) y sus índices

vocabulary = tfidfvect.vocabulary_

# Creamos el diccionario de índices a términos

idx2word = {v: k for k, v in vocabulary.items()}

# Seleccionamos 5 palabras al azar

random.seed(3)  # Usamos una semilla para la reproducibilidad

selected_word_indices = random.sample(range(X_train_T.shape[0]), 5)

# Calculamos la similaridad coseno entre las palabras seleccionadas

for idx in selected_word_indices:
    word = idx2word[idx]
    print(f"\nPalabra: {word}")
    
    # Calculamos la similaridad coseno de la palabra seleccionada con todas las demás palabras

    similarities = cosine_similarity(X_train_T[idx], X_train_T).flatten()
    most_similar_indices = np.argsort(similarities)[::-1][1:6]  # Excluimos la palabra misma
    
    for similar_idx in most_similar_indices:
        similar_word = idx2word[similar_idx]
        similarity_score = similarities[similar_idx]
        print(f"  {similar_word} (similitud: {similarity_score:.4f})")


Palabra: endear
  spurred (similitud: 0.6861)
  doubters (similitud: 0.6020)
  suggestive (similitud: 0.5071)
  burdened (similitud: 0.4720)
  pinhead (similitud: 0.4030)

Palabra: symbolic
  nechvatal (similitud: 0.5850)
  20899 (similitud: 0.5828)
  gaithersburg (similitud: 0.5770)
  allegorical (similitud: 0.4775)
  scalars (similitud: 0.3835)

Palabra: sebastian
  9w (similitud: 0.9452)
  calmly (similitud: 0.5806)
  swerving (similitud: 0.4910)
  sears (similitud: 0.3868)
  lud (similitud: 0.3585)

Palabra: bray
  deals (similitud: 0.4491)
  postage (similitud: 0.3234)
  monkees (similitud: 0.3131)
  hooters (similitud: 0.3131)
  weer (similitud: 0.3131)

Palabra: lenin
  stalin (similitud: 0.5628)
  worshipping (similitud: 0.5615)
  centralization (similitud: 0.5521)
  borderline (similitud: 0.4663)
  maternal (similitud: 0.4303)


#### Análisis de resultados:

- La similitud entre "endear" (hacerse querer) y las palabras relacionadas parece plausible ya que son palabras que podrían aparecer en contextos emocionales o de relaciones.
- "Symbolic" está relacionado con "allegorical", lo cual tiene sentido, ya que ambas palabras se refieren a representaciones simbólicas. Las demás palabras pueden estar relacionadas con nombres propios o términos técnicos.
- "Sebastian" es un nombre propio, por lo que la alta similitud con "9w" puede no ser intuitiva. Las otras palabras parecen menos relacionadas directamente.
- La palabra "bray" puede referirse a un sonido (el rebuzno de un burro) o a un nombre propio, lo cual hace que la similitud con estas palabras no sea claramente evidente.
- "Lenin" y "Stalin" tienen una alta similitud, lo cual es razonable debido a su conexión histórica. "Centralization" también es coherente en este contexto.